# Convert excel into latex table (with style)


1. https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.html
2. https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.to_latex.html

In [3]:
import pandas as pd
from collections import defaultdict, Counter
from tqdm import tqdm 
tqdm.pandas()
import random
from IPython.display import display
import numpy as np
# import matplotlib.pyplot as plt 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_rows', 500)

# from tabulate import tabulate
# from texttable import Texttable

# import latextable

old version: 
```
Requirement already satisfied: pandas in /Users/chunhua/anaconda3/lib/python3.8/site-packages (1.2.4)
Requirement already satisfied: python-dateutil>=2.7.3 in /Users/chunhua/anaconda3/lib/python3.8/site-packages (from pandas) (2.8.1)
Requirement already satisfied: pytz>=2017.3 in /Users/chunhua/anaconda3/lib/python3.8/site-packages (from pandas) (2021.1)
Requirement already satisfied: numpy>=1.16.5 in /Users/chunhua/anaconda3/lib/python3.8/site-packages (from pandas) (1.19.5)
Requirement already satisfied: six>=1.5 in /Users/chunhua/anaconda3/lib/python3.8/site-packages (from python-dateutil>=2.7.3->pandas) (1.15.0)
```

# Packages required to import in latex 
```
# pointers:  https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.to_latex.html
\usepackage{multirow}
\usepackage{booktabs}
\usepackage[table]{xcolor}
\usepackage{siunitx}  #rounding 


\usepackage{etoolbox}
\robustify\bfseries
\sisetup{detect-all = true} (within {document})
\usepackage{etoolbox}
\robustify\itshape
\sisetup{detect-all = true} (within {document})

\usepackage{longtable} 
\usepackage{hyperref}

```

In [34]:
# pd.__version__ 
# 1.2.4
! pip install pandas

In [9]:


class Excel2Latex(object):
    def __init__(self, df, label, caption, output_file, adjustbox_width, precision, column_format):
        self.df = df
        self.label = label 
        self.caption = caption
        self.output_file = output_file
        self.adjustbox = adjustbox_width
        self.precision = precision
        self.column_format = column_format

        self.convert_df_to_latex_table(self.df, label=self.label, caption=self.caption, output_file=self.output_file, adjustbox_width=adjustbox_width, precision=precision, column_format=self.column_format)


    def add_span_columns(self, table):
        ''' 
        add * after table so that the table can span two columns
        '''
        table_lines = table.split("\n")
        table_lines[0] = table_lines[0].replace("table", "table*")
        table_lines[-2] = table_lines[-2].replace("table", "table*")
        return "\n".join(table_lines)

    def move_caption_to_bottom(self, table):
        '''
        the default caption position is on the top, this function shifts it to the bottom
        '''
        table_lines = table.split("\n")
        caption_line_num, end_tabular_line_num = None, None 
        caption_line = None

        for i, line in enumerate(table_lines):
            if line.startswith(r"\caption"):
                caption_line_num = i 
                caption_line = line
            if line.startswith(r'\end{tabular}'):
                end_tabular_line_num = i
        print(caption_line_num)
        print(end_tabular_line_num)

        table_lines.insert(end_tabular_line_num+1, caption_line)
        del table_lines[caption_line_num]
        return "\n".join(table_lines)

    def get_line_with_startswith(self, table_lines, start_str):
        line_num = None
        for i, line in enumerate(table_lines):
            if line.startswith(start_str):
                line_num  = i 
        return line_num


    def write_latex_table(self, table_latex, output_file):
        ''' 
        table_latex: table represented in a list of string
        '''
        with open(output_file, 'w', encoding='utf-8') as fout:
            for line in table_latex:
                fout.write(line)

        print(f"saved {output_file}")

    def adjust_box(self, table_latex, width='columnwidth'):
        ''' 
        This function add the addjust box outside the tabular

        TODO: fill with more options for width
        '''
        table_lines =  table_latex.split("\n")
        if width == 'columnwidth':
            adjust_box_begin = r"\begin{adjustbox}{width=\columnwidth}"

        if width == 'textwidth':
            adjust_box_begin = r"\begin{adjustbox}{width=\textwidth}"

        adjust_box_end = r" \end{adjustbox}"

        tabular_start_line_num = self.get_line_with_startswith(table_lines, r'\begin{tabular}')
        tabular_end_line_num = self.get_line_with_startswith(table_lines, r'\end{tabular}')

        table_lines.insert(tabular_end_line_num +1, adjust_box_end)
        table_lines.insert(tabular_start_line_num-1, adjust_box_begin)

        return "\n".join(table_lines)

    def convert_df_to_latex_table(self, df, label, caption, output_file, adjustbox_width='textwidth', span_two_columns=True, bold_max=True, precision=0, column_format='l'):
        table_string = df.style.highlight_max(
            # props='cellcolor:[HTML]{FFFF00}; color:{red};'
            # props= 'textit:--rwrap; 
            props='textbf:--rwrap;'
        ).format(precision=precision).hide(axis='index')
        
        column_format = column_format* len(df.columns)
        table_latex = table_string.to_latex(
            column_format=column_format, position="!h", position_float="centering",
            hrules=True, 
            multirow_align="t", multicol_align="r",
            caption=caption,
            label=label, 
        )  

        if  span_two_columns:
            table_latex = self.add_span_columns(table_latex)

        # if caption is not None:
        table_latex = self.move_caption_to_bottom(table_latex)
        table_latex = self.adjust_box(table_latex, width=adjustbox_width)

        self.write_latex_table(table_latex, output_file)
        print(table_latex)
        return table_latex 



path = '../files/LabelSWOW.xlsx'
df = pd.read_excel(path, engine="openpyxl", header=[0]) #, header=[0,1])
Excel2Latex(df=df, label='tab:rel_extraion_results', 
            caption='Relation extraction results on human-written explanations', 
            output_file='../files/table_rel_extraction_human.tex',
            adjustbox_width = 'textwidth',
            precision = 1,
            column_format='c'
            )



2
19
saved ../files/table_rel_extraction_human.tex
\begin{table*}[!h]
\centering
\begin{adjustbox}{width=\textwidth}
\label{tab:rel_extraion_results}
\begin{tabular}{cccccccccccccc}
\toprule
Model & Train & P & R & F1 & Acc & P.1 & R.1 & F1.1 & Acc.1 & P.2 & R.2 & F1.2 & Acc.2 \\
\midrule
\textbf{Random} & full & 10.0 & 10.0 & 9.0 & 10.0 & 9.0 & 8.0 & 6.0 & 8.0 & 12.0 & 13.0 & 12.0 & 12.0 \\
Majority class & full & 2.0 & 11.0 & 3.0 & 16.0 & 2.0 & 12.0 & 3.0 & 15.0 & 2.0 & 11.0 & 3.0 & 16.0 \\
LogisticRegression & full & 49.0 & 42.0 & 39.0 & 44.0 & 24.0 & 19.0 & 16.0 & 21.0 & 52.0 & 42.0 & 40.0 & 45.0 \\
ALBERT & \textbf{zero-shot} & 9.0 & 10.0 & 7.0 & 10.0 & 2.0 & 2.0 & 2.0 & 3.0 & 9.0 & 10.0 & 7.0 & 10.0 \\
ALBERT & dev & 47.0 & 46.0 & 44.0 & 53.0 & 22.0 & 29.0 & 20.0 & 26.0 & 48.0 & 46.0 & 44.0 & 55.0 \\
ALBERT & cn & 68.0 & 44.0 & 45.0 & 35.0 & 20.0 & 19.0 & 11.0 & 18.0 & 68.0 & 45.0 & 45.0 & 36.0 \\
ALBERT & full & \textbf{69.0} & \textbf{70.0} & \textbf{69.0} & \textbf{70.0} & 24.

In [15]:
def bold_extreme_values(data, format_string="%.2f", max_=True):
    if max_:
        extrema = data != data.max()
    else:
        extrema = data != data.min()
    bolded = data.apply(lambda x : "\\textbf{%s}" % format_string % x)
    formatted = data.apply(lambda x : format_string % x)
    return formatted.where(extrema, bolded) 


test = pd.DataFrame({"foo" : [0,2,0,0.5,2,1], 
    "bar":[1,1,0,1.5,1,1], 
    "method": ["x","x","x","y","y","y"]},
    index=[0,1,2,0,1,2]).pivot(columns="method")
display(test)
col_show_max = { "foo": True, "bar" : False}
for col in test.columns.get_level_values(0).unique():
    print(col)
    test[col] = test[col].apply(lambda data : bold_extreme_values(data, max_=col_show_max[col]),axis=1)
print(test.to_latex(escape=False))


foo       bar     
method    x    y    x    y
0       0.0  0.5  1.0  1.5
1       2.0  2.0  1.0  1.0
2       0.0  1.0  0.0  1.0

foo
bar
\begin{tabular}{lllll}
\toprule
{} & \multicolumn{2}{l}{foo} & \multicolumn{2}{l}{bar} \\
method &              x &              y &              x &              y \\
\midrule
0 &           0.00 &  \textbf{0.50} &  \textbf{1.00} &           1.50 \\
1 &  \textbf{2.00} &  \textbf{2.00} &  \textbf{1.00} &  \textbf{1.00} \\
2 &           0.00 &  \textbf{1.00} &  \textbf{0.00} &           1.00 \\
\bottomrule
\end{tabular}



In [27]:
path = '../files/LabelSWOW.xlsx'
df = pd.read_excel(path, header=[0,1])

display(df)
df.head()
# model_name={
#     '': '', 
#     '': '',
# }
# df['model'] = df['model'].apply(lambda x: )
print()
print(df.to_latex())

test = df 
col_show_max = { "Test_full": True, "Test_fully_seen" : True, "Test_not_fully_seen":True}

for col in test.columns.get_level_values(0).unique():
    print(col)
    if col in col_show_max:
        test[col] = test[col].apply(lambda data : bold_extreme_values(data, max_=col_show_max[col]),axis=1)
print(test.to_latex(escape=False, multicolumn=True, multirow=True))


Unnamed: 0_level_0 Unnamed: 1_level_0 Test_full                         \
                 Model              Train         P      R Macro-F1    Acc   
0               Random               full      0.10   0.10     0.09   0.10   
1       Majority_class               full      0.02   0.11     0.03   0.16   
2  Logistic Regression               full      0.49   0.42     0.39   0.44   
3               ALBERT          zero-shot      0.09   0.10     0.07   0.10   
4               ALBERT                dev      0.47   0.46     0.44   0.53   
5               ALBERT                 cn      0.68   0.44     0.45   0.35   
6               ALBERT               full      0.69   0.70     0.69   0.70   
7                 BART                dev      3.04   2.35     1.84   2.95   
8                 BART                 cn     61.18  61.04    60.48  62.96   
9                 BART               full     66.56  66.98    66.52  68.85   

  Test_fully_seen                        Test_not_fully_seen                  \
                P      R Macro-F1    Acc                   P      R Macro-F1   
0            0.09   0.08     0.06   0.08                0.12   0.13     0.12   
1            0.02   0.12     0.03   0.15                0.02   0.11     0.03   
2            0.24   0.19     0.16   0.21                0.52   0.42     0.40   
3            0.02   0.02     0.02   0.03                0.09   0.10     0.07   
4            0.22   0.29     0.20   0.26                0.48   0.46     0.44   
5            0.20   0.19     0.11   0.18                0.68   0.45     0.45   
6            0.24   0.31     0.26   0.31                0.72   0.72     0.71   
7            0.00   0.00     0.00   0.00                3.15   2.69     2.02   
8           21.06  21.34    20.37  35.90               62.57  61.94    61.64   
9           34.81  30.44    29.70  35.90               69.30  68.72    68.46   

          
     Acc  
0   0.12  
1   0.16  
2   0.45  
3   0.10  
4   0.55  
5   0.36  
6   0.72  
7   3.11  
8  64.46  
9  70.66


\begin{tabular}{lllrrrrrrrrrrrr}
\toprule
{} &   Unnamed: 0\_level\_0 & Unnamed: 1\_level\_0 & \multicolumn{4}{l}{Test\_full} & \multicolumn{4}{l}{Test\_fully\_seen} & \multicolumn{4}{l}{Test\_not\_fully\_seen} \\
{} &                Model &              Train &         P &      R & Macro-F1 &    Acc &               P &      R & Macro-F1 &    Acc &                   P &      R & Macro-F1 &    Acc \\
\midrule
0 &               Random &               full &      0.10 &   0.10 &     0.09 &   0.10 &            0.09 &   0.08 &     0.06 &   0.08 &                0.12 &   0.13 &     0.12 &   0.12 \\
1 &       Majority\_class &               full &      0.02 &   0.11 &     0.03 &   0.16 &            0.02 &   0.12 &     0.03 &   0.15 &                0.02 &   0.11 &     0.03 &   0.16 \\
2 &  Logistic Regression &               full &      0.49 &   0.42 &     0.39 &   0.44 &            0.24 &   0.19 &     0.16 &   0.21 &                0.52 &   0.42 &     0.40 &   0.45 \\
3 &               ALBE